In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import string
import nltk
import sklearn
import re
# import mglearn as mglearn

from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from bs4 import BeautifulSoup
from string import digits

In [2]:
train = pd.read_csv('train.csv')

train.Summary.fillna('', inplace=True)
train.Text.fillna('', inplace=True)
# train.SumTxt.fillna('', inplace=True)

In [3]:
# train_text, train_score = train['Summary'], train['Score']

In [4]:
# test = pd.read_csv('test.csv')
# test_score = pd.read_csv('labels.csv')

In [5]:
testingbs = train['Text'].loc[1]

In [6]:
testingbs2 = BeautifulSoup(testingbs).get_text(' ')

In [7]:
testingbs

'*****<br />Numi\'s Collection Assortment Melange includes:<br />5 Herbal Teas (caffeine-free, also called "teasans"):<br />* Dry Desert Lime: Lime Herbal Teasan<br />* Fields of Gold: Lemongrass Herbal Teasan<br />* Red Mellow Bush: Rooibos Herbal Teasan<br />* Bushmen\'s Brew: Honeybush Herbal Teasan<br />* Simply Mint: Moroccan Mint<br /><br />2 Green Teas (lower in caffeine):<br />* Temple of Heaven: Gunpowder Green Tea<br />* Monkey King: Jasmine Green Tea<br /><br />2 Black Teas (contain caffeine):<br />* Chinese Breakfast: Yunnan Black Tea<br />* Smoky Tarry: Lapsang Souchong Black Tea<br /><br />This is a total of nine different teas, two tea bags of each one in each box. Numi teas are known for their high-quality, organic and kosher ingredients, and in my opinion, are some of the tastiest and best teas I have ever tried. They do not include artificial ingredients or flavorings.<br /><br />On the box, the manufacturer writes: "From mist-covered mountains to sun-drenched deserts

In [8]:
# train['Text'] = train.apply(lambda x: new_score(x['Score']), axis = 1)

In [9]:
# testingbs2 = testingbs2.translate(str.maketrans('', '', string.punctuation))

In [10]:
testingbs2

'***** Numi\'s Collection Assortment Melange includes: 5 Herbal Teas (caffeine-free, also called "teasans"): * Dry Desert Lime: Lime Herbal Teasan * Fields of Gold: Lemongrass Herbal Teasan * Red Mellow Bush: Rooibos Herbal Teasan * Bushmen\'s Brew: Honeybush Herbal Teasan * Simply Mint: Moroccan Mint 2 Green Teas (lower in caffeine): * Temple of Heaven: Gunpowder Green Tea * Monkey King: Jasmine Green Tea 2 Black Teas (contain caffeine): * Chinese Breakfast: Yunnan Black Tea * Smoky Tarry: Lapsang Souchong Black Tea This is a total of nine different teas, two tea bags of each one in each box. Numi teas are known for their high-quality, organic and kosher ingredients, and in my opinion, are some of the tastiest and best teas I have ever tried. They do not include artificial ingredients or flavorings. On the box, the manufacturer writes: "From mist-covered mountains to sun-drenched deserts to fertile fields, we proudly bring you our tea palette. The flavors range from smooth earthiness,

In [11]:
len(string.punctuation)

32

In [12]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [13]:
string.punctuation[:6] + string.punctuation[7:]#23] + string.punctuation[24:]

'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
len(string.punctuation[:6] + string.punctuation[6:])#23] + string.punctuation[24:])

32

In [15]:
string.punctuation[24:]

']^_`{|}~'

In [16]:
len(string.punctuation)
spaces = ' ' * 32

In [17]:
spaces

'                                '

In [18]:
testingbs2 = testingbs2.translate(str.maketrans(string.punctuation[:6] + string.punctuation[7:],
                                                ' '*len(string.punctuation[:6] + string.punctuation[7:])))

In [19]:
testingbs2

"      Numi's Collection Assortment Melange includes  5 Herbal Teas  caffeine free  also called  teasans      Dry Desert Lime  Lime Herbal Teasan   Fields of Gold  Lemongrass Herbal Teasan   Red Mellow Bush  Rooibos Herbal Teasan   Bushmen's Brew  Honeybush Herbal Teasan   Simply Mint  Moroccan Mint 2 Green Teas  lower in caffeine     Temple of Heaven  Gunpowder Green Tea   Monkey King  Jasmine Green Tea 2 Black Teas  contain caffeine     Chinese Breakfast  Yunnan Black Tea   Smoky Tarry  Lapsang Souchong Black Tea This is a total of nine different teas  two tea bags of each one in each box  Numi teas are known for their high quality  organic and kosher ingredients  and in my opinion  are some of the tastiest and best teas I have ever tried  They do not include artificial ingredients or flavorings  On the box  the manufacturer writes   From mist covered mountains to sun drenched deserts to fertile fields  we proudly bring you our tea palette  The flavors range from smooth earthiness  a

In [20]:
' '.join(testingbs2.split())

"Numi's Collection Assortment Melange includes 5 Herbal Teas caffeine free also called teasans Dry Desert Lime Lime Herbal Teasan Fields of Gold Lemongrass Herbal Teasan Red Mellow Bush Rooibos Herbal Teasan Bushmen's Brew Honeybush Herbal Teasan Simply Mint Moroccan Mint 2 Green Teas lower in caffeine Temple of Heaven Gunpowder Green Tea Monkey King Jasmine Green Tea 2 Black Teas contain caffeine Chinese Breakfast Yunnan Black Tea Smoky Tarry Lapsang Souchong Black Tea This is a total of nine different teas two tea bags of each one in each box Numi teas are known for their high quality organic and kosher ingredients and in my opinion are some of the tastiest and best teas I have ever tried They do not include artificial ingredients or flavorings On the box the manufacturer writes From mist covered mountains to sun drenched deserts to fertile fields we proudly bring you our tea palette The flavors range from smooth earthiness and light floral scents to refreshingly sweet and sour notes

In [21]:
def decontracted(phrase):
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [22]:
decontracted(testingbs2)

'      Numi is Collection Assortment Melange includes  5 Herbal Teas  caffeine free  also called  teasans      Dry Desert Lime  Lime Herbal Teasan   Fields of Gold  Lemongrass Herbal Teasan   Red Mellow Bush  Rooibos Herbal Teasan   Bushmen is Brew  Honeybush Herbal Teasan   Simply Mint  Moroccan Mint 2 Green Teas  lower in caffeine     Temple of Heaven  Gunpowder Green Tea   Monkey King  Jasmine Green Tea 2 Black Teas  contain caffeine     Chinese Breakfast  Yunnan Black Tea   Smoky Tarry  Lapsang Souchong Black Tea This is a total of nine different teas  two tea bags of each one in each box  Numi teas are known for their high quality  organic and kosher ingredients  and in my opinion  are some of the tastiest and best teas I have ever tried  They do not include artificial ingredients or flavorings  On the box  the manufacturer writes   From mist covered mountains to sun drenched deserts to fertile fields  we proudly bring you our tea palette  The flavors range from smooth earthiness 

In [25]:
from string import digits

In [26]:
remove_digits = str.maketrans('', '', digits) 

In [31]:
len(digits)

10

In [29]:
testingbs2.translate(remove_digits)

"      Numi's Collection Assortment Melange includes   Herbal Teas  caffeine free  also called  teasans      Dry Desert Lime  Lime Herbal Teasan   Fields of Gold  Lemongrass Herbal Teasan   Red Mellow Bush  Rooibos Herbal Teasan   Bushmen's Brew  Honeybush Herbal Teasan   Simply Mint  Moroccan Mint  Green Teas  lower in caffeine     Temple of Heaven  Gunpowder Green Tea   Monkey King  Jasmine Green Tea  Black Teas  contain caffeine     Chinese Breakfast  Yunnan Black Tea   Smoky Tarry  Lapsang Souchong Black Tea This is a total of nine different teas  two tea bags of each one in each box  Numi teas are known for their high quality  organic and kosher ingredients  and in my opinion  are some of the tastiest and best teas I have ever tried  They do not include artificial ingredients or flavorings  On the box  the manufacturer writes   From mist covered mountains to sun drenched deserts to fertile fields  we proudly bring you our tea palette  The flavors range from smooth earthiness  and 

In [32]:
testingbs2.lower()

"      numi's collection assortment melange includes  5 herbal teas  caffeine free  also called  teasans      dry desert lime  lime herbal teasan   fields of gold  lemongrass herbal teasan   red mellow bush  rooibos herbal teasan   bushmen's brew  honeybush herbal teasan   simply mint  moroccan mint 2 green teas  lower in caffeine     temple of heaven  gunpowder green tea   monkey king  jasmine green tea 2 black teas  contain caffeine     chinese breakfast  yunnan black tea   smoky tarry  lapsang souchong black tea this is a total of nine different teas  two tea bags of each one in each box  numi teas are known for their high quality  organic and kosher ingredients  and in my opinion  are some of the tastiest and best teas i have ever tried  they do not include artificial ingredients or flavorings  on the box  the manufacturer writes   from mist covered mountains to sun drenched deserts to fertile fields  we proudly bring you our tea palette  the flavors range from smooth earthiness  a

In [23]:
# '@#%$/*'